In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/MNIST/
   creating: data/MNIST/processed/
  inflating: data/MNIST/processed/test.pt  
  inflating: data/MNIST/processed/training.pt  
   creating: data/MNIST/raw/
  inflating: data/MNIST/raw/t10k-images-idx3-ubyte  
  inflating: data/MNIST/raw/t10k-images-idx3-ubyte.gz  
  inflating: data/MNIST/raw/t10k-labels-idx1-ubyte  
 extracting: data/MNIST/raw/t10k-labels-idx1-ubyte.gz  
  inflating: data/MNIST/raw/train-images-idx3-ubyte  
  inflating: data/MNIST/raw/train-images-idx3-ubyte.gz  
  inflating: data/MNIST/raw/train-labels-idx1-ubyte  
  inflating: data/MNIST/raw/train-labels-idx1-ubyte.gz  


### import packages

In [ ]:
import torch
from torch.utils.data import DataLoader
import scipy.misc
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torch import optim

#### Example

In [ ]:
trans_img = transforms.Compose([transforms.ToTensor()])
batch_size = 100
testset = MNIST('./data', train=False, transform=trans_img, download=True)
testloader = DataLoader(testset, batch_size, shuffle=False, num_workers=10)
running_acc = 0.
i = 0
for (img, label) in testloader:
    if i==0:
        print(img.shape)
        print(label)
        i += 1
    else:
        break

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid pote

torch.Size([100, 1, 28, 28])
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9])


### Function

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

'''LeNet in PyTorch.'''
import torch.nn as nn
import torch.nn.functional as F
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding = 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
    def forward(self, x):
        # 100, 1, 28, 28 --> 100, 6, 28, 28        
        out = F.relu(self.conv1(x))
        # 100, 6, 28, 28 --> 100, 6, 14, 14
        out = F.max_pool2d(out, 2)
        # 100, 6, 14, 14 --> 100, 16, 10, 10
        out = F.relu(self.conv2(out))
        # 100, 16, 10, 10 --> 100，16， 5， 5
        out = F.max_pool2d(out, 2)
        # 100，16， 5， 5 --> 100, 16*5*5
        out = out.view(out.size(0), -1) # -1 表示不确定的维度
        # 100, 16*5*5 --> 120
        out = F.relu(self.fc1(out))
        # 120 --> 84
        out = F.relu(self.fc2(out))
        # 84 --> 10
        out = self.fc3(out)
        return out

def train():
    learning_rate = 1e-3
    batch_size = 100
    epoches = 50
    lenet = LeNet()
    trans_img = transforms.Compose([transforms.ToTensor()])
    trainset = MNIST('./data', train=True, transform=trans_img, download=True)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=10)
    criterian = nn.CrossEntropyLoss(reduction='sum')  #loss
    optimizer = optim.SGD(lenet.parameters(), lr=learning_rate)  #optimizer
    lenet.to(device)

    for i in range(epoches):
        running_loss = 0.
        running_acc = 0.
        for (img, label) in trainloader:
            optimizer.zero_grad()  # 求梯度之前对梯度清零以防梯度累加
            output=lenet(img.to(device))
            loss=criterian(output,label.to(device))
            loss.backward()    #loss反传存到相应的变量结构当中
            optimizer.step()   #使用计算好的梯度对参数进行更新
            running_loss+=loss.item()
            #print(output)
            _,predict=torch.max(output,1)
            correct_num=(predict==label.to(device)).sum()
            running_acc+=correct_num.item()

        running_loss/=len(trainset)
        running_acc/=len(trainset)
        print("[%d/%d] Loss: %.5f, Acc: %.2f" % (i + 1, epoches, running_loss,100 * running_acc))

    return lenet


def test(lenet):
    batch_size = 100
    trans_img = transforms.Compose([transforms.ToTensor()])
    testset = MNIST('./data', train=False, transform=trans_img, download=True)
    testloader = DataLoader(testset, batch_size, shuffle=False, num_workers=10)
    running_acc = 0.
    lenet.to(device)
    for (img, label) in testloader:
        output = lenet(img.to(device))
        _, predict = torch.max(output, 1)
        correct_num = (predict == label.to(device)).sum()
        running_acc += correct_num.item()
    running_acc /= len(testset)
    return running_acc

### Train and Test

In [ ]:
if __name__ == '__main__':
    lenet = train()
    torch.save(lenet, 'lenet.pkl') # save model

    lenet = torch.load('lenet.pkl') # load model
    test_acc = test(lenet)
    print("Test Accuracy:Loss: %.2f" % test_acc)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1/50] Loss: 0.59928, Acc: 79.80
[2/50] Loss: 0.08987, Acc: 97.16
[3/50] Loss: 0.06096, Acc: 98.04
[4/50] Loss: 0.04761, Acc: 98.51
[5/50] Loss: 0.03901, Acc: 98.78
[6/50] Loss: 0.03365, Acc: 98.96
[7/50] Loss: 0.02850, Acc: 99.08
[8/50] Loss: 0.02563, Acc: 99.16
[9/50] Loss: 0.02216, Acc: 99.27
[10/50] Loss: 0.01926, Acc: 99.34
[11/50] Loss: 0.01680, Acc: 99.44
[12/50] Loss: 0.01493, Acc: 99.49
[13/50] Loss: 0.01346, Acc: 99.56
[14/50] Loss: 0.01221, Acc: 99.60
[15/50] Loss: 0.01026, Acc: 99.67
[16/50] Loss: 0.00901, Acc: 99.71
[17/50] Loss: 0.00903, Acc: 99.70
[18/50] Loss: 0.00798, Acc: 99.74
[19/50] Loss: 0.00730, Acc: 99.73
[20/50] Loss: 0.00634, Acc: 99.81
[21/50] Loss: 0.00508, Acc: 99.84
[22/50] Loss: 0.00593, Acc: 99.83
[23/50] Loss: 0.00406, Acc: 99.86
[24/50] Loss: 0.00376, Acc: 99.89
[25/50] Loss: 0.00492, Acc: 99.83
[26/50] Loss: 0.00154, Acc: 99.97
[27/50] Loss: 0.00146, Acc: 99.97
[28/50] Loss: 0.00157, Acc: 99.97
[29/50] Loss: 0.00138, Acc: 99.96
[30/50] Loss: 0.00077, 